In [29]:
import requests
import json
import sentencepiece as SentencePiece
from transformers import MarianMTModel, MarianTokenizer
import re
import os

In [30]:
filename = 'results_FCGPT_phi4.json'

data = []
with open(filename) as f:
    data = json.load(f)
data[0]

{'source': 'V roce 1980, nejstarším soudcem Nejvyššího soudu Spojených států byl soudce William O. Douglas. Narodil se 16. října 1898 a sloužil u Nejvyššího soudu od roku 1939 až do jeho odchodu do důchodu v roce 1975. Proto, v roce 1980, soudce Douglas byl stále naživu a byl by nejstarší sloužící spravedlnosti na soudu v té době.',
 'claims': [{'id': 0,
   'claim': 'In 1980, the oldest sitting Supreme Court Justice of the United States was William O. Douglas.',
   'checkworthy': True,
   'checkworthy_reason': 'Yes (This statement provides a specific claim about historical records regarding the age and position of William O. Douglas that can be verified through court records or reliable historical sources.)',
   'origin_text': 'V roce 1980, nejstarším soudcem Nejvyššího soudu Spojených států byl soudce William O. Douglas.',
   'start': 0,
   'end': 95},
  {'id': 1,
   'claim': 'William O. Douglas was born on October 16, 1898.',
   'checkworthy': True,
   'checkworthy_reason': 'Yes (The

In [31]:
# Load the model and tokenizer for English to Czech
model_name = "Helsinki-NLP/opus-mt-en-cs"
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name)

In [32]:
def translate(input):
    inputs = tokenizer(input, return_tensors="pt", padding=True, truncation=True)
    translated = model.generate(**inputs)
    translated_text = tokenizer.decode(translated[0], skip_special_tokens=True)
    return translated_text

for item in data:
    try:
        for claim in item['claims']:
            to_translate = re.search(r'\((.*?)\)', claim['checkworthy_reason']).group(1) if re.search(r'\((.*?)\)', claim['checkworthy_reason']) else claim['checkworthy_reason']
            claim['checkworthy'] = claim['checkworthy_reason'].startswith('Yes')
            translated_text = translate(to_translate)
            #translated_text = to_translate
            claim['checkworthy_reason'] = translated_text
    except:
        print(item)
        # Filter out items that caused errors
        data = [item for item in data if isinstance(item, dict) and 'claims' in item and isinstance(item['claims'], list)]

{'source': 'Chcete-li najít nejlepší interiérové dekoratéry a nejlepší cenu, můžete sledovat tyto kroky: 1. Výzkum: Výzkum on-line najít interiérové dekoratéry ve vaší oblasti. Podívejte se na jejich portfolia, hodnocení zákazníků a hodnocení. 2. Shortlist: Shortlist několik dekoratéři založené na jejich práci, zkušenosti, a recenze. 3. Rozpočet: Určete svůj rozpočet pro projekt a prodiskutujte totéž s každým interiérovým dekoratérem. 4. Porovnat: Porovnejte citace a služby nabízené každým interiérovým dekoratérem. 5. Vyberte: Vyberte si nejlepší interiérový dekoratér, který vyhovuje vašemu stylu, potřebám a rozpočtu.', 'claims': {'To find the best interior decorators and price, follow these steps.': {'text': 'Chcete-li najít nejlepší interiérové dekoratéry a nejlepší cenu, můžete sledovat tyto kroky:', 'start': 0, 'end': 92}, 'Research online to find interior decorators in your area.': {'text': '1. Výzkum: Výzkum on-line najít interiérové dekoratéry ve vaší oblasti.', 'start': 93, 'en

In [33]:
output_filename = os.path.join('./filtered', filename)
with open(output_filename, 'w', encoding='utf-8') as f:
    json.dump(data, f, ensure_ascii=False, indent=2)